In [2]:
from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

In [3]:
def get_sequences_from_file(fasta_fn):
    sequence_data_dict = {}
    for record in SeqIO.parse(fasta_fn, "fasta"):
        description = record.description.split()
        species_name = description[1] + " " + description[2]
        sequence_data_dict[species_name] = record.seq
    return(sequence_data_dict)

In [4]:
with open("seq_get.fasta","w") as f:
        for seq_record in SeqIO.parse("penguins_cytb.fasta", "fasta"):
                f.write(str(seq_record.id) + "\n")
                f.write(str(seq_record.seq[:10]) + "\n")  #first 10 base positions
                f.write(str(seq_record.seq[-10:]) + "\n") #last 10 base positions

In [5]:
f

<_io.TextIOWrapper name='seq_get.fasta' mode='w' encoding='UTF-8'>

In [6]:
penguins_ctyb = SeqIO.parse('penguins_cytb.fasta', 'fasta')
count = 0
for record in penguins_ctyb:
    count +=1
    
count

12

In [7]:
for penguins_ctyb in SeqIO.parse('penguins_cytb.fasta', 'fasta'):
    print(penguins_ctyb.id)  
    print(penguins_ctyb.description)
    print(repr(penguins_ctyb.seq)) # representative sequence, not the entire seq
    print(len(penguins_ctyb))  

AB026282
AB026282 Aptenodytes forsteri mitochondrial gene for cytochrome b, complete cds, haplotype:a
Seq('ATGGCCCCAAATCTCCGAAAATCCCATCCCCTCCTAAAAATAATTAATAACTCC...TAA')
1143
AF076044
AF076044 Aptenodytes patagonicus cytochrome b (cytb) gene, mitochondrial gene encoding mitochondrial protein, complete cds
Seq('ATGGCCCCAAACCTCCGAAAATCCCATCCTCTCCTAAAAATAATTAATAACTCC...TAA')
1143
AF076051
AF076051 Eudyptes chrysocome cytochrome b (cytb) gene, mitochondrial gene encoding mitochondrial protein, complete cds
Seq('ATGGCCCCCAACCTCCGAAAATCCCACCCCCTCCTAAAAACAATCAATAACTCC...TAA')
1143
AF076052
AF076052 Eudyptes chrysolophus cytochrome b (cytb) gene, mitochondrial gene encoding mitochondrial protein, complete cds
Seq('ATGGCCCCCAACCTCCGAAAATCCCACCCCCTCCTAAAAACAATCAATAACTCC...TAA')
1143
AB026277
AB026277 Eudyptes sclateri mitochondrial gene for cytochrome b, complete cds
Seq('ATGGCCCCCAACCTCCGAAAATCCCACCCCCTCCTAAAAACAATCAATAACTCC...TAA')
1143
AB026278
AB026278 Eudyptula minor mitochondrial gene for 

In [18]:
from Bio import SeqIO
from Bio.SeqUtils import GC
import pandas as pd
import os
import glob

In [16]:
def calculate_gc(fasta):
    seq_obj=SeqIO.read(fasta, 'fasta')
    sequence=obj.seq

    gc_content=GC(sequence)
    gc_content=round(gc_content,2)

    filename=os.path.split(fasta)
    filename=filename[1]
    filename=filename.strip('.fasta')

    print(filename)
    print(gc_content)



In [ ]:
for fasta in penguins_ctyb:
    calculate_gc(fasta)

In [23]:
def GC_content(dna_seq):
    count = 0
    for base in dna_seq:
        if base == 'G' or base == 'C':
            count+=1
    perce = count/len("penguins_cytb")*100 # percentatge of GC content
    
    return (perce)

In [25]:
GC_content(penguins_ctyb)

4238.461538461539

## translation